In [15]:
import os
import json
import numpy as np
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.interpolate import interp1d
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score, confusion_matrix

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [16]:
# 랜드마크 인덱스 정의
LANDMARKS = [0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]

def augment_sequence(sequence, factor=0.2):
    time_warped = []
    for landmark in sequence:
        x = np.arange(len(landmark))
        f = interp1d(x, landmark, kind='linear', axis=0)
        x_new = np.linspace(0, len(landmark) - 1, num=int(len(landmark) * (1 + factor)))
        time_warped.append(f(x_new))
    return np.array(time_warped)

# 데이터셋 클래스 정의
class FallSequenceDataset(Dataset):
    def __init__(self, json_files, sequence_length=3):
        self.sequence_length = sequence_length
        self.sequences = []
        self.labels = []
        self.scaler = StandardScaler()

        all_landmarks = []
        
        for json_file in json_files:
            print(f'Processing file: {json_file}')
            with open(json_file, 'r') as f:
                data = json.load(f)
            
            frames = list(data['pose_data'].values())
            
            for i in range(0, len(frames) - self.sequence_length + 1):
                sequence = frames[i:i+self.sequence_length]
                landmarks = []
                fall_frames = 0
                
                for frame in sequence:
                    frame_landmarks = []
                    for landmark in LANDMARKS:
                        frame_landmarks.extend([
                            frame[f'landmark_{landmark}']['x'],
                            frame[f'landmark_{landmark}']['y']
                        ])
                    landmarks.append(frame_landmarks)
                    if frame['class'] == 'Fall':
                        fall_frames += 1
                # 데이터 증강 적용        
                augmented_sequence = augment_sequence(landmarks)            
                all_landmarks.extend(augmented_sequence)
                
                # 레이블 재정의
                if fall_frames == 0:
                    label = 0  # 비낙상
                elif fall_frames == self.sequence_length:
                    label = 2  # 완전 낙상
                else:
                    label = 1  # 낙상 위험
                
                self.sequences.append(augmented_sequence)
                self.labels.append(label)
        
        # 전체 데이터 정규화
        all_landmarks = np.array(all_landmarks)
        all_landmarks_scaled = self.scaler.fit_transform(all_landmarks)
        
        # 정규화된 데이터를 다시 시퀀스로 재구성
        for i in range(len(self.sequences)):
            start = i * self.sequence_length
            end = start + self.sequence_length
            self.sequences[i] = all_landmarks_scaled[start:end]

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return torch.FloatTensor(self.sequences[idx]), torch.LongTensor([self.labels[idx]]).squeeze()

# GRU 모델 정의
class FallDetectionGRU(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, output_size=3, dropout = 0.5):
        super(FallDetectionGRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout = dropout)
        self.fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.dropout(out[ : , -1, :])
        out = self.fc(out)
        return out

# 데이터 로드 및 전처리
json_folder = 'D:\\human_fall\\re_landmark\\re_train_NY_json'
json_files = [os.path.join(json_folder, f) for f in os.listdir(json_folder) if f.endswith('.json')]
dataset = FallSequenceDataset(json_files)

# 데이터셋을 학습 및 검증 세트로 분할
train_indices, val_indices = train_test_split(range(len(dataset)), test_size=0.2)
train_dataset = torch.utils.data.Subset(dataset, train_indices)
val_dataset = torch.utils.data.Subset(dataset, val_indices)

# 데이터 로더 생성 전에 클래스 가중치 계산
class_weights = compute_class_weight('balanced', classes=np.unique(dataset.labels), y=dataset.labels)
class_weights = torch.FloatTensor(class_weights).to(device)

# 손실 함수에 가중치 적용
criterion = nn.CrossEntropyLoss(weight=class_weights)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

if len(dataset) > 0:
    sample_sequence, sample_label = dataset[0]
    input_size = sample_sequence.shape[1]
    
    model = FallDetectionGRU(input_size).to(device)
else:
    print("Dataset is empty")
    exit()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001,  weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
num_epochs = 500
best_loss = float('inf')
patience = 15
no_improve = 0

for epoch in range(num_epochs):
    model.train()
    total_loss_train = 0
    
    for sequences, labels in train_loader:
        sequences, labels = sequences.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(sequences)
        loss_train = criterion(outputs, labels.view(-1))
        loss_train.backward()
        optimizer.step()
        
        total_loss_train += loss_train.item()
    
    avg_loss_train = total_loss_train / len(train_loader)

    # 검증 단계 추가
    model.eval()
    total_loss_val = 0
    
    with torch.no_grad():
        for sequences_val, labels_val in val_loader:
            sequences_val, labels_val = sequences_val.to(device), labels_val.to(device)
            outputs_val = model(sequences_val)
            loss_val = criterion(outputs_val, labels_val.view(-1))
            total_loss_val += loss_val.item()
    
    avg_loss_val = total_loss_val / len(val_loader)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_loss_train:.4f}, Val Loss: {avg_loss_val:.4f}')
    
    scheduler.step(avg_loss_val)
    
    if avg_loss_val < best_loss:
        best_loss = avg_loss_val
        no_improve = 0
        torch.save(model.state_dict(), 'best_fall_detection_gru.pt')
    else:
        no_improve += 1
    
    if no_improve >= patience:
        print("Early stopping")
        break
    
def calculate_metrics(model, data_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for sequences, labels in data_loader:
            sequences, labels = sequences.to(device), labels.to(device)
            outputs = model(sequences)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    f1 = f1_score(all_labels, all_preds, average='weighted')
    cm = confusion_matrix(all_labels, all_preds)
    return f1, cm

# 학습 루프 내에서 성능 지표 계산
train_f1, train_cm = calculate_metrics(model, train_loader)
val_f1, val_cm = calculate_metrics(model, val_loader)
print(f'Train F1: {train_f1:.4f}, Val F1: {val_f1:.4f}')
print(f'Train CM:\n{train_cm}\nVal CM:\n{val_cm}')

print("Training completed")

Processing file: D:\human_fall\re_landmark\re_train_NY_json\00060_H_A_SY_C1.json
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00060_H_A_SY_C2.json
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00060_H_A_SY_C3.json
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00060_H_A_SY_C4.json
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00060_H_A_SY_C5.json
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00060_H_A_SY_C6.json
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00060_H_A_SY_C7.json
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00060_H_A_SY_C8.json
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00093_H_A_FY_C1.json
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00093_H_A_FY_C2.json
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00093_H_A_FY_C3.json
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00093_H_A_FY_C4.json
Processing file: D:\human_fa

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch [1/500], Train Loss: 0.7078, Val Loss: 0.6771
Epoch [2/500], Train Loss: 0.6745, Val Loss: 0.6545
Epoch [3/500], Train Loss: 0.6573, Val Loss: 0.6408
Epoch [4/500], Train Loss: 0.6414, Val Loss: 0.6259
Epoch [5/500], Train Loss: 0.6290, Val Loss: 0.6086
Epoch [6/500], Train Loss: 0.6179, Val Loss: 0.6005
Epoch [7/500], Train Loss: 0.6109, Val Loss: 0.6035
Epoch [8/500], Train Loss: 0.6027, Val Loss: 0.5863
Epoch [9/500], Train Loss: 0.5943, Val Loss: 0.5858
Epoch [10/500], Train Loss: 0.5884, Val Loss: 0.5796
Epoch [11/500], Train Loss: 0.5808, Val Loss: 0.5656
Epoch [12/500], Train Loss: 0.5735, Val Loss: 0.5596
Epoch [13/500], Train Loss: 0.5685, Val Loss: 0.5577
Epoch [14/500], Train Loss: 0.5611, Val Loss: 0.5538
Epoch [15/500], Train Loss: 0.5553, Val Loss: 0.5409
Epoch [16/500], Train Loss: 0.5498, Val Loss: 0.5367
Epoch [17/500], Train Loss: 0.5447, Val Loss: 0.5361
Epoch [18/500], Train Loss: 0.5406, Val Loss: 0.5303
Epoch [19/500], Train Loss: 0.5356, Val Loss: 0.5225
Ep

In [45]:
print(f"JSON 파일 수: {len(json_files)}")
for file in json_files[:5]:  # 처음 5개 파일만 출력
    print(file)

JSON 파일 수: 1464
D:\human_fall\re_landmark\re_train_NY_json\00057_H_A_N_C1.json
D:\human_fall\re_landmark\re_train_NY_json\00057_H_A_N_C2.json
D:\human_fall\re_landmark\re_train_NY_json\00057_H_A_N_C3.json
D:\human_fall\re_landmark\re_train_NY_json\00057_H_A_N_C4.json
D:\human_fall\re_landmark\re_train_NY_json\00057_H_A_N_C5.json


In [42]:
def print_json_structure(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    print(f"File: {json_file}")
    print(f"Keys: {data.keys()}")
    if 'pose_data' in data:
        print(f"Number of frames: {len(data['pose_data'])}")
        print(f"Frame keys: {list(data['pose_data'].values())[0].keys()}")
    else:
        print("No 'pose_data' key found")

# 처음 몇 개의 파일만 확인
for json_file in json_files[:3]:
    print_json_structure(json_file)

File: D:\human_fall\re_landmark\re_train_NY_json\00057_H_A_N_C1.json
Keys: dict_keys(['video_path', 'pose_data'])
Number of frames: 24
Frame keys: dict_keys(['landmark_0', 'landmark_11', 'landmark_12', 'landmark_15', 'landmark_16', 'landmark_23', 'landmark_24', 'landmark_25', 'landmark_26', 'landmark_27', 'landmark_28', 'class', 'confidence'])
File: D:\human_fall\re_landmark\re_train_NY_json\00057_H_A_N_C2.json
Keys: dict_keys(['video_path', 'pose_data'])
Number of frames: 25
Frame keys: dict_keys(['landmark_0', 'landmark_11', 'landmark_12', 'landmark_15', 'landmark_16', 'landmark_23', 'landmark_24', 'landmark_25', 'landmark_26', 'landmark_27', 'landmark_28', 'class', 'confidence'])
File: D:\human_fall\re_landmark\re_train_NY_json\00057_H_A_N_C3.json
Keys: dict_keys(['video_path', 'pose_data'])
Number of frames: 25
Frame keys: dict_keys(['landmark_0', 'landmark_11', 'landmark_12', 'landmark_15', 'landmark_16', 'landmark_23', 'landmark_24', 'landmark_25', 'landmark_26', 'landmark_27', '

In [51]:
print(torch.min(labels), torch.max(labels))

tensor(24) tensor(24)
